# M5 Project Paper Group 1 (MSIS 2537 FALL 2024)

# Data Import and Cleaning

In [1]:
#imports and settings
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
import time as time
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

## Journeys CSV

In [2]:
# Load journey data from CSV
journeys = pd.read_csv('journeys.csv')

# Filter for journeys in San Francisco and remove the city column
journeys = journeys[journeys['Car Parking Address City'] == 'San Francisco'].drop(columns='Car Parking Address City')

# Covert postcode to category
journeys['Car Parking Address Postcode'] = journeys['Car Parking Address Postcode'].astype('category')

# Convert specific columns to datetime format
cols = ['Trip Start At Local Time', 'Trip End At Local Time', 'Trip Created At Local Time']
journeys[cols] = journeys[cols].apply(pd.to_datetime)

# Clean and convert the trip price column to float
journeys.rename(columns={'Trip Sum Trip Price': 'Trip Price'}, inplace=True)
journeys['Trip Price'] = journeys['Trip Price'].str.replace(r"[$,\.]", "", regex=True).astype(float) / 100

# Sort journeys by start time in ascending order
journeys.sort_values(by='Trip Start At Local Time', ascending=True, inplace=True)

journeys

,Trip ID Hash,Car ID Hash,Car Parking Address Postcode,Trip Start At Local Time,Trip End At Local Time,Trip Created At Local Time,Trip Price
14398,41037625705e1c8c54ee2311803da479,9ddbdce433138241891c9799cfb46859,94122,2017-01-01 09:30:00,2017-01-01 17:00:00,2017-01-01 09:28:42,60.00
3968,11b010fd63336457c28d3cd747d673b2,5328fd8804d7560958b4eddfce276139,94117,2017-01-01 10:15:00,2017-01-02 10:00:00,2017-01-01 10:03:48,80.00
20711,5d9ad2b521e99deebf54cbfeeaff9682,a6bc23db01b597f8eee2378305747412,94109,2017-01-01 10:30:00,2017-01-01 15:30:00,2016-12-30 15:58:45,45.00
43917,c7636937599182c1ed7effe0ac8c2f0d,b5e9271dd09182abab262d7c8453d5e5,94115,2017-01-01 10:30:00,2017-01-01 20:30:00,2016-12-31 19:25:54,90.00
27485,7cab48e204def19250c79cbc24ef307a,31daeb68bf1fd988e7dcf6a5976acd49,94110,2017-01-01 10:30:00,2017-01-01 18:30:00,2016-12-31 22:02:50,68.00
...,...,...,...,...,...,...,...
43783,c6d201f4d6e66b0de13598118c55d8ec,046065e6f1433794f9d051bae3c2d850,94112,2017-12-30 22:00:00,2017-12-31 01:00:00,2017-12-30 21:11:54,18.75
19735,59705d88d67a2e60f8ec5084cd9def35,9ddbdce433138241891c9799cfb46859,94122,2017-12-30 22:45:00,2018-01-01 15:30:00,2017-12-30 17:40:03,128.05
3150,0df125971dff90f587bc8de4858d8844,93f34c89e36e37f6187df8708b41ed62,94110,2017-12-30 23:00:00,2017-12-31 04:15:00,2017-12-30 20:05:45,26.25
49454,e0118e8190fc1196ea5df23374c79478,8ebd24949be530b67600a9b11bac7e4e,94103,2017-12-30 23:45:00,2017-12-31 09:30:00,2017-12-30 09:34:41,48.75


## Utilization CSV

In [3]:
# Load CSV file
utilization = pd.read_csv('utilization.csv')

# Convert time column to datetime format
utilization['Car Hourly Utilization Aggregated At Time'] = pd.to_datetime(utilization['Car Hourly Utilization Aggregated At Time'])

# Rename columns for clarity
utilization.rename(columns={
    'Car Hourly Utilization Sum Available Minutes': 'Car Available Minutes',
    'Car Hourly Utilization Sum Utilized Minutes': 'Car Used Minutes'
}, inplace=True)

# Filter rows to include only cars present in the journeys DataFrame
utilization = utilization[utilization['Car ID Hash'].isin(journeys['Car ID Hash'])]

#create utilization % column, drop available and used minutes
utilization['Utilization'] = utilization['Car Used Minutes']/utilization['Car Available Minutes']
utilization.drop(columns = ['Car Used Minutes', 'Car Available Minutes'], inplace = True)

# Sort by time column
utilization = utilization.sort_values(by='Car Hourly Utilization Aggregated At Time')

utilization

<ipython-input-3-70cd859b3962>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utilization['Utilization'] = utilization['Car Used Minutes']/utilization['Car Available Minutes']
<ipython-input-3-70cd859b3962>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utilization.drop(columns = ['Car Used Minutes', 'Car Available Minutes'], inplace = True)


,Car ID Hash,Car Hourly Utilization Aggregated At Time,Utilization
2256014,d8377fc52752d07adee8aacbbf87dbfe,2017-01-01 00:00:00,0.0
1791832,adff5e22a393de23e82444e1abb5215d,2017-01-01 00:00:00,0.0
969142,622e5272a5fc0bbbbbd935b99762447f,2017-01-01 00:00:00,0.0
1817370,b1512d5d9b6140eb341578f80f4e5785,2017-01-01 00:00:00,0.0
955478,6190fdc0e9dd4957ec45360262dc98a0,2017-01-01 00:00:00,0.0
...,...,...,...
1981068,be88e01934f647a731bcd679d77e844e,2017-12-30 23:00:00,0.5
2273483,d88adcb1e702c280716302cb218d4473,2017-12-30 23:00:00,NaN
1022117,643b27320a0a43f302f8cada0fd9a2c5,2017-12-30 23:00:00,1.0
1783096,ad5b0ff5edb01efe41acf0999accf446,2017-12-30 23:00:00,NaN


## Final Dataframe

In [4]:
#all unique cars
car_list = journeys['Car ID Hash'].unique()
car_list = car_list[0:30] #30 cars due to computation challenge

print('Merging and filtering dataframes...')
start_time = time.time()

#create the final combined dataframe
df = pd.DataFrame()

for i in car_list:
    list1 = utilization[utilization['Car ID Hash'] == i]
    list2 = journeys[journeys['Car ID Hash'] == i]

    #perform a merge
    merged = list1.merge(list2, how='left')

    #filter where Utilization is between the start and end times
    merged = merged[(merged["Car Hourly Utilization Aggregated At Time"] >= merged["Trip Start At Local Time"]) &
              (merged["Car Hourly Utilization Aggregated At Time"] <= merged["Trip End At Local Time"])]

    #concat the new dataframe with the complete one
    df = pd.concat([df, merged], ignore_index=True)

#sort df by trip time start
df = df.sort_values(by = 'Car Hourly Utilization Aggregated At Time')

#generate the final dataframe
df2 = pd.DataFrame()

for i in car_list:
    list1 = utilization[utilization['Car ID Hash'] == i]
    list2 = df[df['Car ID Hash'] == i]

    #merge left "utilization" with "df" and duplicate previous row, by each car ID
    df3 = list1.merge(list2, how='left')
    df3 = df3.ffill()

    #concat the new dataframe with the complete one
    df2 = pd.concat([df2, df3], ignore_index=True)

df = df2.copy()

#time calculation
end_time = time.time()
elapsed_time = end_time - start_time

print(f'Done, elapsed time to merge was {round(elapsed_time, 2)} seconds.')

#drop NaN columns (and NaT)
df = df.replace([np.inf, -np.inf], np.nan).dropna()

#clean dataframe
#sort df by Utilization time
df = df.sort_values(by = 'Car Hourly Utilization Aggregated At Time', ignore_index=True)

#create duration column
df['Hours Duration'] = (df['Trip End At Local Time'] - df['Trip Start At Local Time']).dt.total_seconds().astype(float)/3600

#create booking advance time
df['Booking Advance Hours Duration'] = ((df['Trip Start At Local Time'] - df['Trip Created At Local Time'])
                                        .dt.total_seconds().astype(float)/3600).round(4)

#create individual start date and time columns
df[['Start Date','Start Time']] = df['Trip Start At Local Time'].astype(str).str.split(' ', expand = True)
df['Start Date'] = pd.to_datetime(df['Start Date'])
df['Start Time'] = pd.to_datetime(df['Start Time'], format = '%H:%M:%S')

#create Trip Start Day of the week column
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df['Start Day of the Week'] = df['Start Date'].dt.weekday.map(lambda x: days[x])

#create Trip start month column
months = ['January', 'February', 'March', 'April', 'May', 'June',
          'July', 'August', 'September', 'October', 'November', 'December']
df['Start Month'] = df['Start Date'].dt.month.map(lambda x: months[x-1])

#create start hour column
df['Start Hour'] = df['Start Time'].dt.hour

#directly calculate hourly average price per post code
df['Hourly Post Code Average Price'] = df.groupby(['Car Hourly Utilization Aggregated At Time', 'Car Parking Address Postcode'],
                                                  observed = False)['Trip Price'].transform('mean')

#Create trip price to average price per post code column
df['Price to Average Price Per Postcode'] = (df['Trip Price']/df['Hourly Post Code Average Price']).round(2)

#drop columns not needed
df.drop(columns = ['Trip Start At Local Time', 'Trip End At Local Time', 'Trip Created At Local Time',
                   'Start Date', 'Start Time', 'Car Hourly Utilization Aggregated At Time', 'Trip ID Hash',
                  'Hourly Post Code Average Price'], inplace = True)

print(f'\nFinal Dataframe after cleaning is {len(df)} rows long')
print(f'Original Utilization dataframe was {len(utilization)} rows long')

df

Merging and filtering dataframes...
Done, elapsed time to merge was 20.78 seconds.

Final Dataframe after cleaning is 250544 rows long
Original Utilization dataframe was 1328815 rows long


,Car ID Hash,Utilization,Car Parking Address Postcode,Trip Price,Hours Duration,Booking Advance Hours Duration,Start Day of the Week,Start Month,Start Hour,Price to Average Price Per Postcode
0,9ddbdce433138241891c9799cfb46859,1.0,94122,60.00,7.50,0.0217,Sunday,January,9,1.00
1,a6bc23db01b597f8eee2378305747412,1.0,94109,45.00,5.00,42.5208,Sunday,January,10,1.00
2,b5e9271dd09182abab262d7c8453d5e5,1.0,94115,90.00,10.00,15.0683,Sunday,January,10,1.00
3,31daeb68bf1fd988e7dcf6a5976acd49,1.0,94110,68.00,8.00,12.4528,Sunday,January,10,1.00
4,5328fd8804d7560958b4eddfce276139,1.0,94117,80.00,23.75,0.1867,Sunday,January,10,1.00
...,...,...,...,...,...,...,...,...,...,...
250539,a4146ec8abbd83af7efb24e3e8f12c13,0.0,94122,36.64,4.25,0.5908,Saturday,December,12,0.72
250540,9ddbdce433138241891c9799cfb46859,1.0,94122,128.05,40.75,5.0825,Saturday,December,22,2.50
250541,f71f8651e3fa154855271f8ded9601d2,0.0,94102,32.64,3.00,0.2739,Saturday,December,16,0.62
250542,4ee389e31ed04c83df66d1aee2840967,0.0,94122,53.77,8.00,0.0925,Saturday,December,9,1.05


# Environmental Supervised Learning Model

In [5]:
#create training and test data
x = df.drop(columns = 'Utilization')
y = df['Utilization']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state = 42)

x_train

,Car ID Hash,Car Parking Address Postcode,Trip Price,Hours Duration,Booking Advance Hours Duration,Start Day of the Week,Start Month,Start Hour,Price to Average Price Per Postcode
4221,76aa6d08190734decd6ab8f7b573101d,94110,96.00,26.00,1.2458,Saturday,January,10,2.38
92712,b9bad70674669b3ac9d3aad4d681e2f6,94102,11.25,1.25,0.0294,Tuesday,May,9,0.22
120683,b1512d5d9b6140eb341578f80f4e5785,94112,90.00,24.00,3.4503,Saturday,April,15,1.60
133331,4ee389e31ed04c83df66d1aee2840967,94122,93.50,11.00,0.0731,Saturday,July,16,1.09
247203,5328fd8804d7560958b4eddfce276139,94117,72.83,8.00,0.2922,Monday,December,13,1.18
...,...,...,...,...,...,...,...,...,...
119879,001881b594721c5b169be3a2e73c1183,94117,60.75,6.75,0.6800,Sunday,June,10,0.55
103694,b5e9271dd09182abab262d7c8453d5e5,94115,20.25,2.25,19.9894,Sunday,May,14,0.44
131932,d63ded6ed04143a8813121ce0611287c,94112,56.25,6.25,0.1892,Saturday,June,11,0.77
146867,a4146ec8abbd83af7efb24e3e8f12c13,94122,80.00,16.00,20.8528,Friday,July,7,0.93


In [6]:
#categorical and numerical column lists
categorical_list = ['Car ID Hash', 'Car Parking Address Postcode', 'Start Day of the Week', 'Start Month', 'Start Hour', ]
numerical_list = ['Trip Price', 'Hours Duration', 'Booking Advance Hours Duration', 'Price to Average Price Per Postcode']

#standard transformation
preprocessor1 = ColumnTransformer(transformers=[('categorical', OneHotEncoder(sparse_output = False, handle_unknown = 'ignore'), categorical_list),
                                              ('numerical', MinMaxScaler(), numerical_list)], remainder='passthrough')

## Linear Regression

In [7]:
#standard Preprocessor and Linear Regression
time_start = time.time()
model_1 = make_pipeline(preprocessor1, LinearRegression())
model_1.fit(x_train, y_train)
time_end = time.time()
print(f'Train R2 Score: {model_1.score(x_train, y_train):.4f}')
print(f'Test R2 Score: {model_1.score(x_test, y_test):.4f}')
print(f'Seconds to Fit Model: {(time_end - time_start):.3f}')

Train R2 Score: 0.1957
Test R2 Score: 0.2080
Seconds to Fit Model: 2.478


# Reinforcement Learning Models

Notes:
- RL models interact with the SL model.
- Compare Agent's model to the Environment's model

## A/B/N Model

In [8]:
time_start = time.time()

# Initialization
df2 = pd.DataFrame(columns=['Utilization'] + x_train.columns.tolist())
num_iterations = len(x_train)
j = 0
last_time_check = time.time()

# Reinforcement Learning Loop
for i in range(num_iterations):
    # Agent chooses a random price
    agent_price = np.random.uniform(10, 150) #this can be adjusted

    # Get current environment features
    current_features = x_train.iloc[i][x_train.drop(columns = 'Trip Price').columns.tolist()]

    # Predict utilization using supervised learning model #1
    features_with_price = current_features.to_dict()
    features_with_price['Trip Price'] = agent_price
    features_with_price = pd.DataFrame([features_with_price])
    utilization = model_1.predict(features_with_price)[0]

    # Append the agent's data and predicted utilization to DataFrame 2
    new_row = {'Trip Price': agent_price, **current_features.to_dict(), 'Utilization': utilization}
    if df2.empty:
        df2 = pd.DataFrame([new_row])
    else:
        df2 = pd.concat([df2, pd.DataFrame([new_row])], ignore_index=True)

    # Time Check
    j += 1
    if j % 1000 == 0:
        current_time = time.time()
        time_for_last_1000 = current_time - last_time_check
        remaining_iterations = (num_iterations - j) // 1000
        estimated_time_remaining = remaining_iterations * time_for_last_1000 / 60
        last_time_check = current_time
        print(f'Completed loop {j/1000:.0f}k / {num_iterations/1000:.0f}k,',
              f'Estimated Time Remaining: {estimated_time_remaining:.2f} Minutes')

# Train supervised learning model #2 on Agent's DataFrame 2
x2_train = df2.drop(columns = 'Utilization')
y2_train = df2.Utilization

preprocessor2 = ColumnTransformer(transformers=[('categorical', OneHotEncoder(sparse_output = False, handle_unknown = 'ignore'), categorical_list),
                                              ('numerical', MinMaxScaler(), numerical_list)], remainder='passthrough')

model_2 = make_pipeline(preprocessor2, LinearRegression())
model_2.fit(x2_train, y2_train)

time_end = time.time()
print(f'Time to Train RL Model: {(time_end - time_start)/60:.2f} Minutes')

# Compare Agent's Model to the environment's model
environment_train = model_1.score(x_train, y_train)
print(f'Environment Training R2 Score: {environment_train:.4f}')

ABN_train = model_2.score(x_train, y_train)
print(f'A/B/N Training R2 Score: {ABN_train:.4f}')

environment_test = model_1.score(x_test, y_test)
print(f'Environment Test R2 Score: {environment_test:.4f}')

ABN_test = model_2.score(x_test, y_test)
print(f'A/B/N Test R2 Score: {ABN_test:.4f}')

Completed loop 1k / 200k, Estimated Time Remaining: 89.07 Minutes
Completed loop 2k / 200k, Estimated Time Remaining: 79.26 Minutes
Completed loop 3k / 200k, Estimated Time Remaining: 72.70 Minutes
Completed loop 4k / 200k, Estimated Time Remaining: 67.54 Minutes
Completed loop 5k / 200k, Estimated Time Remaining: 61.27 Minutes
Completed loop 6k / 200k, Estimated Time Remaining: 59.72 Minutes
Completed loop 7k / 200k, Estimated Time Remaining: 66.36 Minutes
Completed loop 8k / 200k, Estimated Time Remaining: 63.19 Minutes
Completed loop 9k / 200k, Estimated Time Remaining: 64.26 Minutes
Completed loop 10k / 200k, Estimated Time Remaining: 59.19 Minutes
Completed loop 11k / 200k, Estimated Time Remaining: 58.19 Minutes
Completed loop 12k / 200k, Estimated Time Remaining: 63.28 Minutes
Completed loop 13k / 200k, Estimated Time Remaining: 58.58 Minutes
Completed loop 14k / 200k, Estimated Time Remaining: 57.34 Minutes
Completed loop 15k / 200k, Estimated Time Remaining: 60.82 Minutes
Comp

In [9]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8') #for currency formatting

# Cumulative Reward Calculation and Display for df2
df2['Cumulative Reward'] = df2['Trip Price'] * df2['Utilization']
total_cumulative_reward_df2 = df2['Cumulative Reward'].sum()
formatted_reward = locale.currency(total_cumulative_reward_df2, grouping=True)
print(f"Total Cumulative Reward for df2: {formatted_reward}")

Total Cumulative Reward for df2: $5,349,205.34


## Epsilon Greedy Model

In [10]:
# Epsilon-greedy strategy parameters
epsilon = 0.1  # Exploration rate (10% exploration)
candidate_prices = np.linspace(10, 150, 15)  # Prices to consider during exploitation

# Initialize model_3 (supervised learning model for exploitation)
preprocessor3 = ColumnTransformer(transformers=[('categorical', OneHotEncoder(sparse_output = False, handle_unknown = 'ignore'), categorical_list),
                                              ('numerical', MinMaxScaler(), numerical_list)], remainder='passthrough')

model_3 = make_pipeline(preprocessor3, LinearRegression())
model_3_trained = False

# Initialize
preprocessor1.fit(x_train)
df3 = pd.DataFrame(columns=['Utilization'] + x_train.columns.tolist())
num_iterations = len(x_train)
j = 0
last_time_check = time.time()

# Reinforcement Learning Loop
for i in range(num_iterations):
    # Get current environment features
    current_features = x_train.iloc[i][x_train.drop(columns='Trip Price').columns.tolist()]
    features_with_price = current_features.copy().to_dict()  # Ensure it's a dictionary

    # Epsilon-greedy decision
    if np.random.rand() < epsilon or not model_3_trained:
        # Exploration: Choose a random price
        agent_price = np.random.uniform(10, 150)

    else:
        # Exploitation: Maximize predicted utilization * price
        predicted_rewards = []
        for price in candidate_prices:
            # Add 'Trip Price' to the current features
            features_with_price['Trip Price'] = price
            features_with_price_df = pd.DataFrame([features_with_price])

            # Predict utilization using model_3
            utilization = model_3.predict(features_with_price_df)[0]
            predicted_rewards.append(utilization * price)

        # Select the price that maximizes utilization * price
        agent_price = candidate_prices[np.argmax(predicted_rewards)]

    # Create DataFrame and transform data
    features_with_price = current_features.copy().to_dict()
    features_with_price['Trip Price'] = agent_price
    features_with_price_df = pd.DataFrame([features_with_price])
    data_transformed = model_1.named_steps['columntransformer'].transform(features_with_price_df)

    # Predict utilization using supervised learning model #1
    utilization = model_1.named_steps['linearregression'].predict(data_transformed)[0]

    # Append the agent's data and predicted utilization to DataFrame 3
    new_row = {'Trip Price': agent_price, **current_features.to_dict(), 'Utilization': utilization}
    if df3.empty:
        df3 = pd.DataFrame([new_row])
    else:
        df3 = pd.concat([df3, pd.DataFrame([new_row])], ignore_index=True)

    # Create/Update supervised learning model_3 after collecting enough data, Retrain every 1,000 iterations after 10,000 samples
    if len(df3) > 10000 and i % 1000 == 0:  #this needs to be extended from 10,000 if the data doesn't see every sample (depending on dataset size)
        x3_train = df3.drop(columns=['Utilization'])
        y3_train = df3['Utilization']
        model_3.fit(x3_train, y3_train)
        model_3_trained = True

    j += 1
    if j % 1000 == 0:
        current_time = time.time()
        time_for_last_1000 = current_time - last_time_check
        remaining_iterations = (num_iterations - j) // 1000
        estimated_time_remaining = remaining_iterations * time_for_last_1000 / 60
        last_time_check = current_time
        print(f'Completed loop {j/1000:.0f}k / {num_iterations/1000:.0f}k,',
              f'Estimated Time Remaining: {estimated_time_remaining:.2f} Minutes')

# Train model one last time at the end
x3_train = df3.drop(columns=['Utilization'])
y3_train = df3['Utilization']
model_3.fit(x3_train, y3_train)

time_end = time.time()
print(f'Time to Train RL Model: {(time_end - time_start)/60:.2f} Minutes')

# Compare Agent's Model to the environment's model
environment_train = model_1.score(x_train, y_train)
print(f'Environment Training R2 Score: {environment_train:.4f}')

EGreedy_train = model_3.score(x_train, y_train)
print(f'EGreedy Training R2 Score: {EGreedy_train:.4f}')

environment_test = model_1.score(x_test, y_test)
print(f'Environment Test R2 Score: {environment_test:.4f}')

EGreedy_test = model_3.score(x_test, y_test)
print(f'EGreedy Test R2 Score: {EGreedy_test:.4f}')

Completed loop 1k / 200k, Estimated Time Remaining: 62.49 Minutes
Completed loop 2k / 200k, Estimated Time Remaining: 67.17 Minutes
Completed loop 3k / 200k, Estimated Time Remaining: 60.95 Minutes
Completed loop 4k / 200k, Estimated Time Remaining: 60.22 Minutes
Completed loop 5k / 200k, Estimated Time Remaining: 63.23 Minutes
Completed loop 6k / 200k, Estimated Time Remaining: 60.84 Minutes
Completed loop 7k / 200k, Estimated Time Remaining: 59.37 Minutes
Completed loop 8k / 200k, Estimated Time Remaining: 64.02 Minutes
Completed loop 9k / 200k, Estimated Time Remaining: 59.19 Minutes
Completed loop 10k / 200k, Estimated Time Remaining: 58.90 Minutes
Completed loop 11k / 200k, Estimated Time Remaining: 318.12 Minutes
Completed loop 12k / 200k, Estimated Time Remaining: 306.99 Minutes
Completed loop 13k / 200k, Estimated Time Remaining: 330.03 Minutes
Completed loop 14k / 200k, Estimated Time Remaining: 308.70 Minutes
Completed loop 15k / 200k, Estimated Time Remaining: 312.96 Minutes

In [11]:
# Cumulative reward calculation and display for df3
df3['Cumulative Reward'] = df3['Trip Price'] * df3['Utilization']
total_cumulative_reward_df3 = df3['Cumulative Reward'].sum()
formatted_reward_df3 = locale.currency(total_cumulative_reward_df3, grouping=True)
print(f"Total Cumulative Reward for df3: {formatted_reward_df3}")

Total Cumulative Reward for df3: $11,314,715.16


### Summary

In [15]:
#ABN model time and cumulative Reward
print(f"ABN Training Time: 75.76 Minutes") #hard coding due to overwrite from E-greedy
print(f"Total Cumulative Reward for df2: {formatted_reward}")

#Epsilon greedy model time and cumulative reward
EGreedy_total_time = (time_end - time_start) / 60  #time in minutes
print(f"Epsilon Greedy Training Time: {EGreedy_total_time:.2f} minutes")
print(f"Total Cumulative Reward for df3: {formatted_reward_df3}")

ABN Training Time: 75.76 Minutes
Total Cumulative Reward for df2: $5,349,205.34
Epsilon Greedy Training Time: 414.81 minutes
Total Cumulative Reward for df3: $11,314,715.16


# Feature Importance of the Three models

In [13]:
#model 1
model_1_feature_names = model_1.named_steps['columntransformer'].get_feature_names_out()
model_1_coefficients = model_1.named_steps['linearregression'].coef_
model_1_coef_df = pd.DataFrame({'Feature': model_1_feature_names, 'Environment Coefficients': model_1_coefficients})
model_1_coef_df['Environment Absolute Coefficients'] = model_1_coef_df['Environment Coefficients'].abs()

#model 2
model_2_feature_names = model_2.named_steps['columntransformer'].get_feature_names_out()
model_2_coefficients = model_2.named_steps['linearregression'].coef_
model_2_coef_df = pd.DataFrame({'Feature': model_2_feature_names, 'A/B/N Coefficients': model_2_coefficients})
model_2_coef_df['A/B/N Absolute Coefficients'] = model_2_coef_df['A/B/N Coefficients'].abs()

#model 3
model_3_feature_names = model_3.named_steps['columntransformer'].get_feature_names_out()
model_3_coefficients = model_3.named_steps['linearregression'].coef_
model_3_coef_df = pd.DataFrame({'Feature': model_3_feature_names, 'EGreedy Coefficients': model_3_coefficients})
model_3_coef_df['EGreedy Absolute Coefficients'] = model_3_coef_df['EGreedy Coefficients'].abs()

#combined Dataframe for all coefficients
merged = pd.merge(model_1_coef_df, model_2_coef_df, on='Feature')
all_model_features = pd.merge(merged, model_3_coef_df, on='Feature')
all_model_features = all_model_features.sort_values(by='Environment Absolute Coefficients', ascending=False)

cols = all_model_features.columns[1:]
for col in cols:
    all_model_features[col] = all_model_features[col].apply(lambda x: float("{:.2e}".format(x)))

pd.options.display.max_rows = None
all_model_features

,Feature,Environment Coefficients,Environment Absolute Coefficients,A/B/N Coefficients,A/B/N Absolute Coefficients,EGreedy Coefficients,EGreedy Absolute Coefficients
26,categorical__Car ID Hash_d719907bc88b06eadc7ef...,-5.810000e+12,5.810000e+12,4.140000e+09,4.140000e+09,4.500000e+09,4.500000e+09
23,categorical__Car ID Hash_c0050d79423575ee7e2b3...,-5.810000e+12,5.810000e+12,4.140000e+09,4.140000e+09,4.500000e+09,4.500000e+09
12,categorical__Car ID Hash_5cfeba13d65aaea9e7462...,-5.810000e+12,5.810000e+12,4.140000e+09,4.140000e+09,4.500000e+09,4.500000e+09
40,categorical__Car Parking Address Postcode_94133,5.100000e+12,5.100000e+12,-3.340000e+09,3.340000e+09,-1.860000e+09,1.860000e+09
28,categorical__Car ID Hash_e913321219505fb67071c...,-2.820000e+12,2.820000e+12,-2.230000e+08,2.230000e+08,2.080000e+09,2.080000e+09
22,categorical__Car ID Hash_b9bad70674669b3ac9d3a...,-2.820000e+12,2.820000e+12,-2.230000e+08,2.230000e+08,2.080000e+09,2.080000e+09
4,categorical__Car ID Hash_081ba0bd4491bc7958a36...,-2.820000e+12,2.820000e+12,-2.230000e+08,2.230000e+08,2.080000e+09,2.080000e+09
29,categorical__Car ID Hash_f71f8651e3fa154855271...,-2.820000e+12,2.820000e+12,-2.230000e+08,2.230000e+08,2.080000e+09,2.080000e+09
6,categorical__Car ID Hash_26c6519fa6cb24389eed6...,-2.800000e+12,2.800000e+12,1.170000e+09,1.170000e+09,3.870000e+09,3.870000e+09
13,categorical__Car ID Hash_6c3e1812633ac44d768d9...,-2.800000e+12,2.800000e+12,1.170000e+09,1.170000e+09,3.870000e+09,3.870000e+09
